In [1]:
from elasticsearch import Elasticsearch, RequestError, TransportError
from elasticsearch.helpers import scan
import couchdb
import json
import pickle
from time import sleep
from collections import defaultdict

In [2]:
# Set ES connection
es = Elasticsearch('http://jb049:32801')
print es.ping()
es_local = Elasticsearch('http://localhost:9200')

es_local.ping()

True


True

In [3]:
# Set types
doc_types=['assessment', 'profile', 'occurrence', 'taxon']
#doc_types=['assessment', 'profile', 'taxon']
query = json.loads('{"query": {"match_all": {}}}')

In [9]:
# Get indexes
indices = es.indices.get_aliases()
try:
    del indices['.kibana']
except KeyError:
    pass
#print indices.keys()
indices = ['plantas_raras_do_cerrado']

i = defaultdict(dict)
items_with_id = defaultdict(dict)
items_with_extreme = defaultdict(dict)

#indices = ['pan_espinhaco_meridional']
#doc_types = ['occurrence']
#break_flag = False

for index in indices:
    i[index] = defaultdict(int)
    items_with_id[index] = defaultdict(list)
    items_with_extreme[index] = defaultdict(list)
    
    for doc_type in doc_types:
        print "Recuperando %s de %s e inserindo no ES local" % (doc_type, index)
        results = scan(es, index=index, doc_type=doc_type, query=query)      

        for item in results:
            # Check for docs with _id and _rev fields
            try:
                del item['_source']['_id']
                del item['_source']['_rev']
                items_with_id[index][doc_type].append(item['_id'])
            except KeyError:
                pass
            
            # Check for docs with different extremeFluctuation mapping
            try:
                population = item['_source']['population']
                try:
                    extremeFluctuation = population['extremeFluctuation']['properties']
                except TypeError:
                    extremeFluctuation = population['extremeFluctuation']
                    item['_source']['population']['extremeFluctuation'] = {"extremeFluctuation": extremeFluctuation}
                    items_with_extreme[index][doc_type].append(item['_id'])
            except KeyError:
                pass
            
            # Remove taxon.scientificNameWithoutAuthoship
            try:
                del item['_source']['taxon']['scientificNameWithoutAuthoship']
            except KeyError:
                pass
            
            # Remove taxon.synonyms
            try:
                del item['_source']['taxon']['synonyms']
            except KeyError:
                pass
                
            try:
                es_local.index(index=index, doc_type=doc_type, body=item['_source'])
                i[index][doc_type]+=1
            except RequestError as err:
                print err
                print item
                #break_flag = True

            #if break_flag:
            #    break
            #if not (i[index][doc_type] % 50000):
                #print i[index][doc_type]
        print "Inseridos %d itens do tipo %s do indice %s" % (i[index][doc_type], doc_type, index)

Recuperando assessment de plantas_raras_do_cerrado e inserindo no ES local
Inseridos 605 itens do tipo assessment do indice plantas_raras_do_cerrado
Recuperando profile de plantas_raras_do_cerrado e inserindo no ES local
Inseridos 615 itens do tipo profile do indice plantas_raras_do_cerrado
Recuperando occurrence de plantas_raras_do_cerrado e inserindo no ES local
Inseridos 14346 itens do tipo occurrence do indice plantas_raras_do_cerrado
Recuperando taxon de plantas_raras_do_cerrado e inserindo no ES local
Inseridos 634 itens do tipo taxon do indice plantas_raras_do_cerrado


In [29]:
import pickle
pickle.dump( i, open( "inserted.pkl", "wb" ) )
pickle.dump( items_with_id, open( "items_with_id.pkl", "wb" ) )
pickle.dump( items_with_extreme, open( "items_with_extreme.pkl", "wb" ) )

In [18]:
import pandas

In [35]:
df = pandas.DataFrame.from_dict(i)

In [36]:
print df

            endemicas_rio_de_janeiro  especies_indicadas  livro_vermelho_2013  \
assessment                       403                   8                 4617   
occurrence                     16281                 489               309482   
profile                         1043                  34                 5263   
taxon                           1584                  67                15714   

            novas_especies  outros  pan_bacia_alto_tocantins  \
assessment               7       0                         7   
occurrence              61      22                       783   
profile                 10       2                        10   
taxon                    7      25                        34   

            pan_espinhaco_meridional  pan_grao_mogol  \
assessment                         0               0   
occurrence                      8362             259   
profile                            0               0   
taxon                            131             

In [40]:
print df.sum()
print "Total:", df.sum().sum()

endemicas_rio_de_janeiro     19311
especies_indicadas             598
livro_vermelho_2013         335076
novas_especies                  85
outros                          49
pan_bacia_alto_tocantins       834
pan_espinhaco_meridional      8493
pan_grao_mogol                 283
plantas_raras_do_cerrado     16200
public                      335076
dtype: int64
Total: 716005
